In [ ]:
#  Description: Safe Money Transfer with Transaction, Savepoint, and Rollback in MySQL

import mysql.connector

try:
    #  1. Connect to MySQL database
    connection = mysql.connector.connect(
        host="localhost",
        user="your_username",       # Replace with your MySQL username
        password="your_password",   # Replace with your MySQL password
        database="your_database"    # Replace with your DB name (must contain 'accounts' table)
    )
    cursor = connection.cursor()

    #  2. Start the transaction manually
    connection.start_transaction()

    #  3. Deduct ₹500 from Account 1 (Sender)
    cursor.execute("""
        UPDATE accounts SET balance = balance - 500
        WHERE account_id = 1
    """)

    #  4. Savepoint before crediting receiver
    cursor.execute("SAVEPOINT after_deduction")

    #  5. Try to credit ₹500 to Account 2 (Receiver)
    cursor.execute("""
        UPDATE accounts SET balance = balance + 500
        WHERE account_id = 2
    """)
    rows_affected = cursor.rowcount

    #  6. If success — COMMIT
    if rows_affected == 1:
        connection.commit()
        print("\U00002705 Transfer successful!")

    #  7. If failed — rollback and refund
    else:
        print("\u26a0\ufe0f Receiver not found. Reverting transaction...")

        cursor.execute("ROLLBACK TO after_deduction")
        cursor.execute("""
            UPDATE accounts SET balance = balance + 500
            WHERE account_id = 1
        """)
        connection.commit()
        print(" Transfer failed. Amount refunded to sender.")

except mysql.connector.Error as e:
    #  8. On any other DB error — rollback everything
    connection.rollback()
    print(" Database error occurred:", e)

finally:
    #  9. Cleanup
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("🖚 Connection closed.")
